In [13]:
import pandas as pd
import numpy as np
import os
import sys
import sklearn
import missingno as msno

In [14]:
raw_df = pd.read_csv('./mimic_cut_3d.csv').set_index('stay_id').drop(columns=['Unnamed: 0'])
raw_df.head(n=10)

,creatinine,fraction_inspired_oxygen,lactate,urine_output,diastolic_blood_pressure,systolic_blood_pressure,gcs,partial_pressure_of_oxygen,heart_rate,temperature,respiratory_rate
stay_id,,,,,,,,,,,
33751102.0,1.150000,0.880000,6.140000,280.000000,57.058333,111.058333,15.0,98.562500,112.104167,36.303175,25.824811
33751102.0,1.150000,0.833333,6.650000,257.142857,56.162963,109.051852,15.0,97.714286,110.759259,36.465278,22.955387
33751102.0,1.150000,0.833333,7.500000,232.500000,55.746667,107.546667,15.0,103.166667,109.683333,36.558025,23.659848
33751102.0,1.166667,0.833333,8.200000,232.500000,55.587879,106.951515,15.0,103.166667,108.712121,36.558025,24.236226
33751102.0,1.166667,0.800000,8.200000,231.666667,55.538889,107.288889,15.0,103.166667,107.486111,36.572222,24.716540
33751102.0,1.150000,0.766667,8.200000,219.000000,54.319444,106.736111,15.0,108.600000,105.569444,36.719444,25.340278
35520609.0,1.575000,0.500000,1.966667,55.181818,76.717172,120.684343,15.0,62.750000,130.594017,38.074074,22.940171
35520609.0,1.633333,0.533333,1.700000,53.700000,76.194444,120.638889,15.0,72.250000,129.388889,37.977778,23.388889
35520609.0,1.633333,0.533333,1.700000,57.200000,76.833333,122.083333,15.0,72.250000,128.416667,37.977778,23.833333


In [15]:
def mask_random_data(data, mask_fraction=0.1):
    """
    Randomly masks a given fraction of the data in a DataFrame.

    :param data: Pandas DataFrame to be masked.
    :param mask_fraction: Fraction of the data to be masked. Default is 0.1 (10%).
    :return: DataFrame with masked data.
    """
    # Flatten the DataFrame to work with individual elements
    flattened_data = data.to_numpy().flatten()

    # Calculate the number of elements to mask
    num_to_mask = int(len(flattened_data) * mask_fraction)

    # Randomly select indices to mask
    mask_indices = np.random.choice(len(flattened_data), num_to_mask, replace=False)

    # Mask the selected indices
    for idx in mask_indices:
        flattened_data[idx] = np.nan

    # Reshape the data back to its original shape and create a new DataFrame
    return pd.DataFrame(flattened_data.reshape(data.shape), columns=data.columns)

In [16]:
mu, std = raw_df.mean(axis=0).values, raw_df.std(axis=0).values
normalized_df = (raw_df - mu)/std
masked_df = mask_random_data(normalized_df)
masked_df.head(n=10)

,creatinine,fraction_inspired_oxygen,lactate,urine_output,diastolic_blood_pressure,systolic_blood_pressure,gcs,partial_pressure_of_oxygen,heart_rate,temperature,respiratory_rate
0,-0.543570,1.732144,0.701248,2.429507,0.108413,0.432360,0.264455,-0.579785,0.955194,-0.728371,0.953613
1,-0.543570,1.463702,0.857793,2.149416,0.005184,NaN,0.264455,-0.594241,0.886037,-0.541851,NaN
2,-0.543570,1.463702,1.118701,1.847442,-0.042812,0.113009,0.264455,-0.501313,0.830711,-0.435134,0.507925
3,-0.532450,1.463702,1.333567,1.847442,-0.061119,0.058886,0.264455,NaN,0.780770,-0.435134,0.626581
4,NaN,1.271958,1.333567,1.837230,NaN,0.089567,0.264455,-0.501313,0.717727,-0.418798,NaN
5,-0.543570,1.080214,1.333567,1.682013,NaN,0.039297,0.264455,-0.408710,0.619170,-0.249401,0.853865
6,-0.260024,-0.453738,-0.579760,-0.325415,NaN,1.307749,0.264455,-1.190158,1.905966,1.309268,0.359770
7,-0.221106,-0.261994,NaN,-0.343573,2.314649,1.303615,0.264455,-1.028244,NaN,1.198468,0.452144
8,-0.221106,-0.261994,-0.661613,-0.300684,2.388307,1.434973,0.264455,-1.028244,1.794004,1.198468,0.543640
9,-0.221106,-0.261994,-0.661613,-0.270049,2.119293,1.230358,0.264455,-1.077955,1.738298,1.245345,0.526484


In [17]:
masked_df = mask_random_data(normalized_df)
print(masked_df.isna().sum())
na_indices = masked_df.isna().to_numpy()
cf_imputed = masked_df.fillna(method='ffill').fillna(method='bfill').to_numpy()
mean_imputed = masked_df.fillna(masked_df.mean()).to_numpy()
median_imputed = masked_df.fillna(masked_df.median()).to_numpy()

creatinine                    322
fraction_inspired_oxygen      329
lactate                       317
urine_output                  322
diastolic_blood_pressure      298
systolic_blood_pressure       307
gcs                           288
partial_pressure_of_oxygen    292
heart_rate                    284
temperature                   315
respiratory_rate              298
dtype: int64


In [18]:
def RMSD_global(y, pred, na_indices):
    results = []
    assert y.shape == pred.shape
    assert y.shape == na_indices.shape
    for i in range(y.shape[1]):
        y_i, pred_i = y[:, i][na_indices[:, i]], pred[:, i][na_indices[:, i]]
        #print(y_i, pred_i)
        var = np.square(y_i - pred_i) 
        rmsd = np.sum(var) / y_i.shape[0]
        results.append(rmsd)
    return np.array(results)



In [19]:
y = normalized_df.to_numpy()
cf_rmsd = RMSD_global(y, cf_imputed, na_indices)
mean_rmsd = RMSD_global(y, mean_imputed, na_indices)
median_rmsd = RMSD_global(y, median_imputed, na_indices)

In [20]:
print(np.mean(cf_rmsd))
print(np.mean(mean_rmsd))
print(np.mean(median_rmsd))

0.39715055127877635
0.9643908043599763
1.0099991529216585
